In [15]:
!pip install transformers

In [16]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.callbacks import EarlyStopping, ModelCheckpoint
import torch

In [17]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [18]:
import pandas as pd

def load_drugs_data(file_path):
    """Loads the 'drugs.csv' dataset into a pandas DataFrame.

    Args:
        file_path (str): Path to the CSV file.

    Returns:
        pd.DataFrame: The loaded dataset.
    """

    data = pd.read_csv(file_path)
    return data


In [19]:
data = load_drugs_data("/content/drugs.csv")

In [21]:
data = data.iloc[:20,]

In [22]:
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Load stopwords
stop_words = set(stopwords.words('english'))

# Function to remove stopwords, punctuation, and other unnecessary words
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords, punctuation, and other unnecessary words
    filtered_tokens = [word.lower() for word in tokens if (word.lower() not in stop_words and word.lower() not in string.punctuation)]

    # Join the filtered tokens back into a single string
    filtered_text = ' '.join(filtered_tokens)

    return filtered_text

# Apply the preprocessing function to the 'text' column
data['description'] = data['description'].apply(preprocess_text)

# Display the DataFrame with processed text
print(data)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


                                          description                  name
0   generic name aripiprazole oral ar-i-pip-ra-zol...               Abilify
1   generic name aripiprazole lauroxil injection a...      Abilify Maintena
2   generic name abiraterone acetate a-bir-a-te-ro...           Abiraterone
3   pronunciation a-seet-a-min-oh-fen generic name...         Acetaminophen
4   generic name acetylcysteine oral a-seet-il-sis...        Acetylcysteine
5   pronunciation ac-tem-ra generic name tocilizum...               Actemra
6   pronunciation ak-tōs generic name pioglitazone...                 Actos
7   generic name acyclovir oral a-sye-klo-veer bra...             Acyclovir
8   pronunciation add-ur-all generic name amphetam...              Adderall
9   generic name amphetamine dextroamphetamine am-...           Adderall XR
10  generic name fluticasone salmeterol floo-tik-a...         Advair Diskus
11  generic name ibuprofen eye-bue-proe-fen brand ...                 Advil
12  pronunci

In [45]:
import pandas as pd
from nltk.tokenize import word_tokenize

# Assuming you already have a DataFrame named 'df' with a column named 'text' containing the text data

# Function to extract the first 400 words from the text
def extract_first_400_words(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Extract the first 400 words
    first_400_words = ' '.join(tokens[:200])

    return first_400_words

# Apply the function to the 'text' column
data['description'] = data['description'].apply(extract_first_400_words)

# Display the DataFrame with the extracted words
print(data)


                                          description                  name
0   generic name aripiprazole oral ar-i-pip-ra-zol...               Abilify
1   generic name aripiprazole lauroxil injection a...      Abilify Maintena
2   generic name abiraterone acetate a-bir-a-te-ro...           Abiraterone
3   pronunciation a-seet-a-min-oh-fen generic name...         Acetaminophen
4   generic name acetylcysteine oral a-seet-il-sis...        Acetylcysteine
5   pronunciation ac-tem-ra generic name tocilizum...               Actemra
6   pronunciation ak-tōs generic name pioglitazone...                 Actos
7   generic name acyclovir oral a-sye-klo-veer bra...             Acyclovir
8   pronunciation add-ur-all generic name amphetam...              Adderall
9   generic name amphetamine dextroamphetamine am-...           Adderall XR
10  generic name fluticasone salmeterol floo-tik-a...         Advair Diskus
11  generic name ibuprofen eye-bue-proe-fen brand ...                 Advil
12  pronunci

In [46]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [47]:
len(data.iloc[0,0])

1511

In [48]:
!pip install datasets

In [49]:
from datasets import Dataset
dataset = Dataset.from_pandas(data)


In [50]:
dataset

Dataset({
    features: ['description', 'name'],
    num_rows: 20
})

In [51]:
def tokenize(batch):
  return tokenizer(batch["description"], padding=True, truncation=True)

In [52]:
encoded = dataset.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [53]:
len(encoded)

20

In [54]:
import tensorflow as tf
from datasets import Dataset

# Assuming you have already loaded your dataset and encoded emotions as emotions_encoded
# emotions_encoded is a Dataset object containing features: ['description', 'name', 'input_ids', 'token_type_ids', 'attention_mask']

# Define batch size
BATCH_SIZE = 64

# Define a function to order the dataset
def order(inp):
    return {
        "input_ids": inp["input_ids"],
        "attention_mask": inp["attention_mask"],
        "token_type_ids": inp["token_type_ids"]
    }, inp["description"]

# Split the dataset into train and test sets
train_size = int(0.8 * len(encoded))
train_data = encoded[:train_size]
test_data = encoded[train_size:]

# Create TensorFlow datasets from the train and test data
train_dataset = tf.data.Dataset.from_tensor_slices(train_data)
test_dataset = tf.data.Dataset.from_tensor_slices(test_data)

# Shuffle and batch the train dataset
train_dataset = train_dataset.shuffle(1000).batch(BATCH_SIZE)

# Apply the order function to the train dataset
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# Batch the test dataset
test_dataset = test_dataset.batch(BATCH_SIZE)


In [55]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
tokenized_data = tokenizer(dataset["description"], return_tensors="np", padding=True)
# Tokenizer returns a BatchEncoding, but we convert that to a dict for Keras
tokenized_data = dict(tokenized_data)

labels = np.array(dataset["name"])  # Label is already an array of 0 and 1

In [56]:
class BERTForClassification(tf.keras.Model):
  def __init__(self, bert_model, num_classes):
    super().__init__()
    self.bert = bert_model
    self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

  def call(self, inputs):
    x = self.bert(inputs)[1]
    return self.fc(x)

In [58]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [59]:
classifier = BERTForClassification(model, num_classes=6)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

In [60]:
history = classifier.fit(
    train_dataset,
    epochs=3
)